<a href="https://colab.research.google.com/github/marxinal/Multimodal-MRS-Master-Thesis/blob/main/Models_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
%cd /content/drive/MyDrive/Thesis Data Last

/content/drive/MyDrive/Thesis Data Last


# 1. Loading Libraries

In [ ]:
import numpy as np
from numpy import load
import pandas as pd
import keras
import matplotlib.pyplot as plt
import random
from keras.models import Sequential
from keras import initializers
from keras import optimizers
from keras.utils.vis_utils import plot_model
from keras.layers import *
from keras.utils import np_utils
import re
import glob
import cv2
import pickle
from sklearn.preprocessing import LabelEncoder
from tempfile import TemporaryFile
from sklearn.model_selection import train_test_split
import pandas as pd 
from numpy import savetxt
from numpy import asarray
from numpy import savez_compressed
from numpy import savetxt
from numpy import asarray
from numpy import savez_compressed
from numpy import mask_indices
import re
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TFDistilBertModel
from keras import regularizers

In [ ]:
%%script echo skipping 

# Split the dataset
# Loading the finalized data frame
all_songs_selected_2 = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/all_songs_selected_2.xlsx")
# Create a path to sliced spectrograms in Google Drive
all_songs_selected_2["drive_path"] = ""
path = "/content/drive/MyDrive/Thesis Data Last/spectrogram_cut/"
counter = 0 

for i in np.arange(0, len (all_songs_selected_2)):
  file_name = all_songs_selected_2.filename[i]
  song = file_name.replace(".mp3", "")
  counter = i
  genre = str(all_songs_selected_2['tag'][i])
  all_songs_selected_2["drive_path"].loc[i] = str(path + genre + str(counter) + "_" + "sliced.jpg")

all_songs_selected_2.to_excel("/content/drive/MyDrive/Thesis Data Last/all_songs_selected_2.xlsx")

# Creating the train and test data from the total df
train, test = train_test_split(all_songs_selected_2, test_size = 0.2) 
# Creating the train and the validation test set from the training data
train, val = train_test_split(train, test_size = 0.1) 
# Printing to see if we actually got DataFrames from our split
print(type(train), type(test), type(val)) 

# Writing data splits to shared folder
val.to_excel("/content/drive/MyDrive/Thesis Data Last/val_data.xlsx")
test.to_excel("/content/drive/MyDrive/Thesis Data Last/test_data.xlsx")
train.to_excel("/content/drive/MyDrive/Thesis Data Last/train_data.xlsx")



skipping


## 3.1 Audio-Only Model (CNN) 

In [ ]:
# Load the datasets
val_data  = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/val_data.xlsx")
test_data  = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/test_data.xlsx")
train_data  = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/train_data.xlsx")

enc = LabelEncoder()
val_data['labels'] = enc.fit_transform(val_data['tag'])
test_data['labels'] = enc.transform(test_data['tag'])
train_data['labels'] = enc.transform(train_data['tag'])

# Get sparse labels output
train_genre = train_data['labels'] 
test_genre = test_data['labels']
val_genre = val_data['labels']

                       

### 3.1.1 Data Processing (Val, Train, Test): Images

In [ ]:
# Load the sliced images from the validation set
path = "/content/drive/MyDrive/Thesis Data Last/spectrogram_cut/"

val_data  = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/val_data.xlsx")

# Validation Set
images_val = []

# Make image input for the model
for i in np.arange(0, len (val_data)):
  temp = cv2.imread(val_data.drive_path[i], cv2.IMREAD_UNCHANGED)
  images_val.append(cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY))

In [ ]:
# Load the sliced images from the train set
path = "/content/drive/MyDrive/Thesis Data Last/spectrogram_cut/"

train_data  = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/train_data.xlsx")

# Train Set
images_train = []

for i in np.arange(0, len (train_data)):
  temp = cv2.imread(train_data.drive_path[i], cv2.IMREAD_UNCHANGED)
  images_train.append(cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY))

In [ ]:
# Load the sliced images from the test set
path = "/content/drive/MyDrive/Thesis Data Last/spectrogram_cut/"

test_data  = pd.read_excel("/content/drive/MyDrive/Thesis Data Last/test_data.xlsx")

# Test set
images_test = []
song_name = []

for i in np.arange(0, len (test_data)):
  temp = cv2.imread(test_data.drive_path[i], cv2.IMREAD_UNCHANGED)
  images_test.append(cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY))
  file_name = test_data.filename[i]
  song_names = file_name.replace(".mp3", "")
  song_name.append(song_names)


In [ ]:
# Saving the computations to the Drive to skip computing them again
savez_compressed('/content/drive/MyDrive/Thesis Data Last/test_images.npz', images_test)
savez_compressed('/content/drive/MyDrive/Thesis Data Last/song_names.npz', song_name)
savez_compressed('/content/drive/MyDrive/Thesis Data Last/train_images.npz', images_train)
savez_compressed('/content/drive/MyDrive/Thesis Data Last/val_images.npz', images_val)

In [ ]:
# Convert to arrays to input in the model
images_val = np.array(images_val)
images_test = np.array(images_test)
images_train = np.array(images_train)

# Expand the dimensions of the image to have a channel dimension. (nx128x128) ==> (nx128x128x1)
train_x = images_train.reshape(images_train.shape[0], images_train.shape[1], images_train.shape[2], 1)
test_x = images_test.reshape(images_test.shape[0], images_test.shape[1], images_test.shape[2], 1)
val_x = images_val.reshape(images_val.shape[0], images_val.shape[1], images_val.shape[2], 1)

# Normalize the matrices.
train_x = train_x / 255.
test_x = test_x / 255.
val_x = val_x /255

In [ ]:
# Saving the computations to the Drive to skip computing them again
savez_compressed('val.npz', val_x)
savez_compressed('train.npz', train_x)
savez_compressed('test.npz', test_x)
savez_compressed('val_test.npz', val_genre)
savez_compressed('train_test.npz', train_genre)
savez_compressed('test_test.npz', test_genre)

### 3.1.2 Run the Audio-only Model


In [ ]:
# Create the CNN model

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=[7,7], kernel_initializer = initializers.he_normal(seed=1), activation="relu", input_shape=(128,128,1)))
# Dim = (122x122x64)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (61x61x64)
model.add(Conv2D(filters=128, kernel_size=[7,7], strides=2, kernel_initializer = initializers.he_normal(seed=1), activation="relu"))
# Dim = (28x28x128)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (14x14x128)
model.add(Conv2D(filters=256, kernel_size=[3,3], kernel_initializer = initializers.he_normal(seed=1), activation="relu"))
# Dim = (12x12x256)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (6x6x256)
model.add(Conv2D(filters=512, kernel_size=[3,3], kernel_initializer = initializers.he_normal(seed=1), activation="relu"))
# Dim = (4x4x512)
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=[2,2], strides=2))
# Dim = (2x2x512)
model.add(BatchNormalization())
model.add(Flatten())
# Dim = (2048)
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(1024, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (1024)
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (256)
model.add(Dropout(0.25))
model.add(Dense(64, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (64)
model.add(Dense(32, activation="relu", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (32)
model.add(Dense(4, activation="softmax", kernel_initializer=initializers.he_normal(seed=1)))
# Dim = (8)
print (model.summary())
plot_model(model, to_file="/content/drive/MyDrive/Thesis Data Last/audio_saved_model/Model_Architecture_Audio.jpg")
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr=0.00005), metrics=['accuracy'])

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 122, 122, 64)      3200      
                                                                 
 batch_normalization_90 (Bat  (None, 122, 122, 64)     256       
 chNormalization)                                                
                                                                 
 average_pooling2d_60 (Avera  (None, 61, 61, 64)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_61 (Conv2D)          (None, 28, 28, 128)       401536    
                                                                 
 batch_normalization_91 (Bat  (None, 28, 28, 128)      512       
 chNormalization)                                                
                                                     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Fit and save the audio-only (CNN) model
pd.DataFrame(model.fit(train_x, train_genre, epochs=15, verbose=1, batch_size = 16, callbacks = [callback], validation_data = (val_x, val_genre)).history).to_csv("/content/drive/MyDrive/Thesis Data Last/audio_saved_model/training_history_audio.csv")
score = model.evaluate(test_x, test_genre, verbose=1)
print (score)
model.save("/content/drive/MyDrive/Thesis Data Last/audio_saved_model/Model_Audio.h5")


Epoch 1/15
819/819 [==============================] - 21s 24ms/step - loss: 1.6682 - accuracy: 0.3337 - val_loss: 1.2401 - val_accuracy: 0.4382
Epoch 2/15
819/819 [==============================] - 20s 24ms/step - loss: 1.3226 - accuracy: 0.3848 - val_loss: 1.1824 - val_accuracy: 0.4670
Epoch 3/15
819/819 [==============================] - 19s 24ms/step - loss: 1.2543 - accuracy: 0.4113 - val_loss: 1.1233 - val_accuracy: 0.5027
Epoch 4/15
819/819 [==============================] - 19s 23ms/step - loss: 1.2002 - accuracy: 0.4402 - val_loss: 1.0933 - val_accuracy: 0.5137
Epoch 5/15
819/819 [==============================] - 19s 23ms/step - loss: 1.1472 - accuracy: 0.4743 - val_loss: 1.0520 - val_accuracy: 0.5247
Epoch 6/15
819/819 [==============================] - 19s 24ms/step - loss: 1.1214 - accuracy: 0.4826 - val_loss: 1.0349 - val_accuracy: 0.5405
Epoch 7/15
819/819 [==============================] - 19s 23ms/step - loss: 1.0976 - accuracy: 0.5007 - val_loss: 1.0062 - val_accuracy:

## 3.2 Multimodal Network

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 79.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### 3.2.1 Data Processing (Val, Train, Test): Text

In [ ]:
# Get Distilled Bert model and tokenizer
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
# Get text (lyrics) data BERT input
# Remove info from brackets
def remove_brackets(data):
    for text in np.arange(0, len(data)):
        data.lyrics.loc[text] = re.sub("[\(\[].*?[\)\]]", "", data.lyrics[text])
    return data.lyrics

# Exchange /n for space
def get_space(data):
  for text in np.arange(0, len(data)):
    data.lyrics.loc[text] = data.lyrics.loc[text].replace("\n", " ")
  return data.lyrics

In [ ]:
# Execute the functions
train_data.lyrics = remove_brackets(train_data)
test_data.lyrics = remove_brackets(test_data)
val_data.lyrics = remove_brackets(val_data)

train_data.lyrics = get_space(train_data)
test_data.lyrics = get_space(test_data)
val_data.lyrics = get_space(val_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# Generate BERT input (ids, masks)
X_trainD_input_ids = np.zeros((len(train_data), 512))
X_trainD_attn_masks = np.zeros((len(train_data), 512))

X_testD_input_ids = np.zeros((len(test_data), 512))
X_testD_attn_masks = np.zeros((len(test_data), 512))

X_valD_input_ids = np.zeros((len(val_data), 512))
X_valD_attn_masks = np.zeros((len(val_data), 512))


def generate_dstil_data(df, ids, masks, dbert_tokenizer):
    for i, text in tqdm(enumerate(df.lyrics)):
        tokenized_text = dbert_tokenizer.encode_plus(
            text,
            max_length=512, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
# Execute the function to execute the data
X_trainD_input_ids,X_trainD_attn_masks = generate_dstil_data(train_data, X_trainD_input_ids, X_trainD_attn_masks, dbert_tokenizer)
X_testD_input_ids,X_testD_attn_masks = generate_dstil_data(test_data, X_testD_input_ids, X_testD_attn_masks, dbert_tokenizer)
X_valD_input_ids,X_valD_attn_masks = generate_dstil_data(val_data, X_valD_input_ids, X_valD_attn_masks, dbert_tokenizer)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

### 3.2.2 Run the Multimodal (Audio-Lyrical) Model

In [ ]:
# Define the Multimodal model 
def mod_full (run = True):
    model = Sequential()
    # CNN input
    cnn_input = Input(shape=(128, 128, 1))
    c = Conv2D(filters=64, kernel_size=[7,7], kernel_initializer = initializers.he_normal(seed=1), activation="relu", input_shape=(128,128,1)) (cnn_input)
# Dim = (122x122x64)
    c = BatchNormalization()(c)
    c = AveragePooling2D(pool_size=[2,2], strides=2) (c)
# Dim = (61x61x64)
    c = Conv2D(filters=128, kernel_size=[7,7], strides=2, kernel_initializer = initializers.he_normal(seed=1), activation="relu") (c)
# Dim = (28x28x128)
    c = BatchNormalization()(c)
    c = AveragePooling2D(pool_size=[2,2], strides=2) (c)
# Dim = (14x14x128)
    c = Conv2D(filters=256, kernel_size=[3,3], kernel_initializer = initializers.he_normal(seed=1), activation="relu") (c)
# Dim = (12x12x256)
    c = BatchNormalization() (c)
    c = AveragePooling2D(pool_size=[2,2], strides=2) (c)
    # Dim = (6x6x256)
    c = Conv2D(filters=512, kernel_size=[3,3], kernel_initializer = initializers.he_normal(seed=1), activation="relu") (c)
# Dim = (4x4x512)
    c = BatchNormalization() (c)
    c = AveragePooling2D(pool_size=[2,2], strides=2) (c)
# Dim = (2x2x512)
    c = BatchNormalization() (c)
    c = Flatten() (c)
# Dim = (2048)
    c = BatchNormalization() (c)
    c = Dropout(0.6) (c)
# Dim = (1024)
    c = Dense(1024, activation="relu", kernel_initializer=initializers.he_normal(seed=1)) (c)
    c = Dropout(0.5) (c)
    c = Dense(256, activation="relu", kernel_initializer=initializers.he_normal(seed=1)) (c)
    c = Dropout(0.25) (c)
    cnn_out = Dense(64, activation="relu", kernel_initializer=initializers.he_normal(seed=1)) (c)
    
# Bert Input
    dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    # defining 2 input layers for input_ids and attn_masks
    input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(512), name='attention_mask', dtype='int32')

    bert_embds = dbert_model(input_ids, attention_mask=attn_masks)[0][:,0,:] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
    bert_out = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
    
# Concatenate 
    input_final = concatenate([cnn_out, bert_out])

# Output genre
    r = Dense(32, activation = 'relu')(input_final)
    r = Dropout(0.3)(r)
    output_genre = Dense(4, activation="softmax", kernel_initializer=initializers.he_normal(seed=1), kernel_regularizer=regularizers.l2(0.01)) (r)
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    model = tf.keras.models.Model(inputs=[cnn_input, input_ids, attn_masks], outputs = output_genre)
    return model

In [ ]:
# Run and save the model
model = mod_full(run = True)

plot_model(model, to_file="/content/drive/MyDrive/Thesis Data Last/full_saved_model/Full_Model_Architecture.jpg")
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr=0.0001), metrics=['sparse_categorical_accuracy'])
pd.DataFrame(model.fit([train_x, X_trainD_input_ids, X_trainD_attn_masks], train_genre, epochs=5, verbose=1, batch_size = 16, validation_data = ([val_x, X_valD_input_ids, X_valD_attn_masks], val_genre)).history).to_csv("/content/drive/MyDrive/Thesis Data Last/full_saved_model/training_history_full.csv")
score = model.evaluate([test_x, X_testD_input_ids, X_testD_attn_masks], test_genre, verbose=1)
print (score)
model.save("/content/drive/MyDrive/Thesis Data Last/full_saved_model/Model_Full.h5")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: U

Epoch 1/5
819/819 [==============================] - 481s 581ms/step - loss: 1.0057 - sparse_categorical_accuracy: 0.5780 - val_loss: 0.8567 - val_sparse_categorical_accuracy: 0.6751
Epoch 2/5
819/819 [==============================] - 475s 580ms/step - loss: 0.7814 - sparse_categorical_accuracy: 0.7092 - val_loss: 0.7352 - val_sparse_categorical_accuracy: 0.7184
Epoch 3/5
819/819 [==============================] - 475s 580ms/step - loss: 0.5886 - sparse_categorical_accuracy: 0.7968 - val_loss: 0.7912 - val_sparse_categorical_accuracy: 0.7115
Epoch 4/5
819/819 [==============================] - 475s 580ms/step - loss: 0.4306 - sparse_categorical_accuracy: 0.8658 - val_loss: 0.9678 - val_sparse_categorical_accuracy: 0.6964
Epoch 5/5
114/114 [==============================] - 43s 362ms/step - loss: 1.0624 - sparse_categorical_accuracy: 0.6832
[1.06236732006073, 0.6832417845726013]
